# Problem 3.b. - DC Location and Transportation

In [1]:
import pandas as pd
import os
from pyomo.environ import *
from pyomo.opt import SolverFactory
os.chdir(r"C:\Users\User\Desktop\BUS 730 - Prescriptive Modeling & Optimization for BA\Assignments\HW3_Integer (MIP) Optimization")
%pprint

Pretty printing has been turned OFF


In [2]:
# reading in DC data (capacity and annual fixed operating cost)
DCdata = pd.read_excel(open('P3_DCLocation.xlsx','rb'), sheet_name='DCdata') 
DCdata

,location,capacity,fixedcost
0,Durham,20655,3831700
1,Fort Wayne,11279,1848400
2,St. Petersburg,25376,2190000
3,Laredo,40169,1361300
4,Buffalo,25167,1312000
5,Madison,12428,2168300
6,Lubbock,37810,1323800
7,Chandler,23909,1432500
8,Scottsdale,5493,1386600
9,Reno,25840,1577700


In [3]:
# converting capacity to a list
capacity = DCdata["capacity"].tolist()
capacity

[20655, 11279, 25376, 40169, 25167, 12428, 37810, 23909, 5493, 25840, 28462, 23012, 41854, 14660, 49261, 22358, 6008, 5526, 30026, 17341, 21896, 49754, 45177]

In [4]:
# converting fixedcost to a list
fixedcost = DCdata["fixedcost"].tolist()
fixedcost

[3831700, 1848400, 2190000, 1361300, 1312000, 2168300, 1323800, 1432500, 1386600, 1577700, 1303100, 1117800, 2856900, 1393600, 1249100, 1236600, 1275700, 1840500, 1452100, 2114800, 2393800, 1266000, 1484700]

In [5]:
# converting locations of DCs to a list
DCs = DCdata["location"].tolist()
DCs

['Durham', 'Fort Wayne', 'St. Petersburg', 'Laredo', 'Buffalo', 'Madison', 'Lubbock', 'Chandler', 'Scottsdale', 'Reno', 'Glendale', 'Norfolk', 'Winston–Salem', 'North Las Vegas', 'Gilbert', 'Chesapeake', 'Irving', 'Hialeah', 'Garland', 'Fremont', 'Richmond', 'Boise', 'Baton Rouge']

In [6]:
# reading in city data (annual demand)
citydata = pd.read_excel(open('P3_DCLocation.xlsx','rb'), sheet_name='citydata') 
citydata

,city,demand
0,New York,8622
1,Los Angeles,3999
2,Chicago,2716
3,Houston,3284
4,Phoenix,1626
...,...,...
71,Orlando,517
72,Irvine,338
73,Toledo,357
74,Jersey City,569


In [7]:
# converting demand to a list
demand = citydata["demand"].tolist()
demand

[8622, 3999, 2716, 3284, 1626, 1580, 1511, 1419, 1341, 1035, 950, 892, 884, 879, 874, 863, 859, 724, 901, 693, 685, 874, 683, 773, 620, 414, 351, 672, 440, 779, 633, 416, 768, 444, 791, 614, 742, 793, 553, 324, 463, 577, 388, 575, 483, 371, 425, 638, 318, 580, 496, 756, 357, 654, 783, 755, 730, 555, 524, 663, 419, 537, 494, 431, 371, 665, 671, 795, 752, 496, 646, 517, 338, 357, 569, 371]

In [8]:
# converting names of cities to a list
cities = citydata["city"].tolist()
cities

['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix', 'Philadelphia', 'San Antonio', 'San Diego', 'Dallas', 'San Jose', 'Austin', 'Jacksonville', 'San Francisco', 'Columbus', 'Fort Worth', 'Indianapolis', 'Charlotte', 'Seattle', 'Denver', 'Washington, D.C.', 'Boston', 'El Paso', 'Detroit', 'Nashville', 'Memphis', 'Portland', 'Oklahoma City', 'Las Vegas', 'Louisville', 'Baltimore', 'Milwaukee', 'Albuquerque', 'Tucson', 'Fresno', 'Sacramento', 'Mesa', 'Kansas City', 'Atlanta', 'Long Beach', 'Omaha', 'Raleigh', 'Colorado Springs', 'Miami', 'Virginia Beach', 'Oakland', 'Minneapolis', 'Tulsa', 'Arlington', 'New Orleans', 'Wichita', 'Cleveland', 'Tampa', 'Bakersfield', 'Aurora', 'Anaheim', 'Honolulu', 'Santa Ana', 'Riverside', 'Corpus Christi', 'Lexington', 'Stockton', 'St. Louis', 'Saint Paul', 'Henderson', 'Pittsburgh', 'Cincinnati', 'Anchorage', 'Greensboro', 'Plano', 'Newark', 'Lincoln', 'Orlando', 'Irvine', 'Toledo', 'Jersey City', 'Chula Vista']

In [9]:
# reading in distribution cost data
distcostdata = pd.read_excel(open('P3_DCLocation.xlsx','rb'), sheet_name='distcost') 
distcostdata

,Unnamed: 0,New York,Los Angeles,Chicago,Houston,Phoenix,Philadelphia,San Antonio,San Diego,Dallas,...,Anchorage,Greensboro,Plano,Newark,Lincoln,Orlando,Irvine,Toledo,Jersey City,Chula Vista
0,Durham,2693,1752,1820,2585,2861,3446,3814,3100,2098,...,2240,2169,2906,1592,3903,1351,3545,2851,1936,3710
1,Fort Wayne,2690,2581,1851,2175,1851,1895,2529,2525,1872,...,1042,2098,2770,1614,835,2652,694,294,1702,1341
2,St. Petersburg,2480,3128,2782,2213,2837,3252,2588,2735,1111,...,1521,2142,2392,500,1800,1270,1132,597,1730,3385
3,Laredo,5144,5693,3046,5593,4414,4492,5348,5757,4681,...,5324,5901,4510,3718,3165,5423,3204,3431,5895,4124
4,Buffalo,3436,4475,4340,5177,4286,4186,4459,4863,5337,...,5642,5111,5504,4217,4031,4532,5300,5761,4460,4240
5,Madison,3587,1973,2033,3632,935,1516,3159,3690,1811,...,3327,1242,2413,2880,2368,902,3515,2353,3686,1423
6,Lubbock,2404,794,2885,998,2791,1674,1414,133,2143,...,47,486,2743,1955,1302,500,1397,1173,1758,1622
7,Chandler,2805,2738,2750,3852,2442,3986,2293,2814,4996,...,3935,4184,4501,2995,3109,2585,5113,4117,2191,3099
8,Scottsdale,5486,5742,3576,4881,5403,3433,4993,5424,4196,...,3477,3824,5751,5908,5520,5247,3234,3079,3536,3174
9,Reno,1940,3552,3835,2819,3095,3953,2891,3931,3382,...,3509,4581,3216,3402,2129,2490,2408,3922,3355,2751


In [10]:
# converting distribution cost to a list of lists
distcost = distcostdata.loc[:, 'New York':'Chula Vista'].values.tolist()
distcost[0] # display the first DC list

[2693, 1752, 1820, 2585, 2861, 3446, 3814, 3100, 2098, 2113, 3576, 2312, 3298, 2423, 3268, 1754, 1433, 2504, 2132, 1368, 3656, 3182, 2368, 3817, 1970, 2430, 1896, 2058, 2076, 3375, 2721, 1831, 3345, 1245, 1920, 3707, 1249, 2722, 3883, 1670, 1036, 2156, 2447, 3725, 2589, 3766, 2144, 2100, 980, 2584, 1690, 3946, 3390, 2062, 3830, 3705, 2660, 2099, 1897, 1008, 3041, 1284, 1903, 1716, 2956, 3253, 2240, 2169, 2906, 1592, 3903, 1351, 3545, 2851, 1936, 3710]

In [11]:
# declare a concrete model
model = ConcreteModel()

# declare the decision variables
n_DC = len(DCs)
n_city = len(cities)

model.x = Var(range(n_DC), domain = Binary)  # Locate a DC at that location? (n_DC*1)
model.y = Var(range(n_DC), range(n_city), domain = NonNegativeReals)  # Units distributed from DC to retail city (n_DC*n_city)

# i refers to the DC Location
# j refers to the city lacation

# specify the objective: Minimize total costs (fixed operating expenses plus distribution costs)
totalfixed = sum(fixedcost[i]*model.x[i] for i in range(n_DC)) 
totaldist = sum(model.y[i,j]*distcost[i][j] for i in range(n_DC) for j in range(n_city))
model.Objective = Objective(expr = totalfixed + totaldist, sense= minimize) 

# Constraint: Units distributed from DC cannot exceed DC Capacity
model.DCCapacity =  ConstraintList() 
for i in range(n_DC): # for each DC location
    model.DCCapacity.add(expr = sum(model.y[i,j] for j in range(n_city)) <= capacity[i])

# Constraint: Must meet required demand needs in each retail city 
model.DemandNeeds =  ConstraintList() 
for j in range(n_city): # for each city
    model.DemandNeeds.add(expr = sum(model.y[i,j] for i in range(n_DC)) >= demand[j])

# Constraint: Only can distribute units from DC to retail city if a DC exists there
model.MaxDistribute =  ConstraintList() 
for i in range(n_DC): # for each DC location
    model.MaxDistribute.add(expr = sum(model.y[i,j] for j in range(n_city)) <= model.x[i]*10000000)

# specify the solver and solve
opt = SolverFactory('glpk')
results = opt.solve(model)#, tee = True)

In [12]:
# print the solution

# Total costs
print("Total costs (fixed operating and distribution costs):", model.Objective()) # 58,006,498

# print the DCs

numDC = 0
locDC = []
for i in range(n_DC):
    if model.x[i]() == 1:
        numDC += model.x[i]()
        locDC.append(DCs[i])

print("\nThe number of distribution centers:", numDC)  # 5
print("\nDistribution centers located in:", locDC)
# ['Lubbock', 'Glendale', 'Winston–Salem', 'Irving', 'Fremont']

# print how units are distributed from DCs to retail cities
print("\nThe number of units distributed: \n")

for i in range(n_DC):   # for each DC
    if model.x[i]() == 1:
        output = ""
        for j in range(n_city):  # for each retail city
            if model.y[i,j]() != 0:
                output = output + str(model.y[i,j]()) + " units to " + str(cities[j]) + ". "             
        
        print("DC:", DCs[i], "distributes:", output, "\n")

Total costs (fixed operating and distribution costs): 58006498.0

The number of distribution centers: 5.0

Distribution centers located in: ['Lubbock', 'Glendale', 'Winston–Salem', 'Irving', 'Fremont']

The number of units distributed: 

DC: Lubbock distributes: 3999.0 units to Los Angeles. 3284.0 units to Houston. 1419.0 units to San Diego. 1035.0 units to San Jose. 863.0 units to Indianapolis. 724.0 units to Seattle. 693.0 units to Washington, D.C.. 620.0 units to Memphis. 672.0 units to Las Vegas. 416.0 units to Albuquerque. 768.0 units to Tucson. 614.0 units to Mesa. 742.0 units to Kansas City. 553.0 units to Long Beach. 324.0 units to Omaha. 463.0 units to Raleigh. 577.0 units to Colorado Springs. 371.0 units to Minneapolis. 318.0 units to New Orleans. 580.0 units to Wichita. 756.0 units to Tampa. 654.0 units to Aurora. 783.0 units to Anaheim. 524.0 units to Corpus Christi. 419.0 units to Stockton. 665.0 units to Cincinnati. 671.0 units to Anchorage. 795.0 units to Greensboro.  



In [13]:
# save the solution to a dataframe
solution = []
for i in range(n_DC):  # for each DC
    if model.x[i]() == 1:
        DCOutput = []
        DCOutput.append(DCs[i])
        for j in range(n_city):  # for each retail city
            DCOutput.append(model.y[i,j]())
        solution.append(DCOutput)
        
cities.insert(0, "DC")
df_sol = pd.DataFrame(solution, columns = cities)
df_sol

,DC,New York,Los Angeles,Chicago,Houston,Phoenix,Philadelphia,San Antonio,San Diego,Dallas,...,Anchorage,Greensboro,Plano,Newark,Lincoln,Orlando,Irvine,Toledo,Jersey City,Chula Vista
0,Lubbock,0.0,3999.0,0.0,3284.0,0.0,0.0,0.0,1419.0,0.0,...,671.0,795.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Glendale,0.0,0.0,0.0,0.0,1626.0,0.0,1511.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,517.0,0.0,0.0,569.0,0.0
2,Winston–Salem,0.0,0.0,2716.0,0.0,0.0,0.0,0.0,0.0,1341.0,...,0.0,0.0,0.0,496.0,0.0,0.0,0.0,357.0,0.0,371.0
3,Irving,2722.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,752.0,0.0,0.0,0.0,338.0,0.0,0.0,0.0
4,Fremont,5900.0,0.0,0.0,0.0,0.0,1580.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,646.0,0.0,0.0,0.0,0.0,0.0
